<a href="https://colab.research.google.com/github/ttogle918/AI_practice/blob/main/QA%20task/03_BERT_QA_korsquad_BertModel%EB%A1%9C%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# korsquad를 사용하여 QA task 풀기(BertModel fine-tuning)

참고 사이트

[huggingface QA 설명](https://huggingface.co/course/chapter7/7?fw=tf)

[huggingface git : ModelOutput](https://github.com/huggingface/transformers/blob/v4.21.0/src/transformers/utils/generic.py#L147)

[huggingface git : QuestionAnsweringModelOutput](https://github.com/huggingface/transformers/blob/a9eee2ffecc874df7dd635b2c6abb246fdb318cc/src/transformers/modeling_outputs.py#L764)

[huggingface docs](https://huggingface.co/docs/transformers/model_doc/bert)

In [ ]:
!pip install transformers
!pip install datasets

In [1]:
from datasets import load_dataset, load_metric#, list_metrics

from transformers import BertModel, AutoTokenizer, BertConfig, BertPreTrainedModel
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
from transformers.modeling_outputs import QuestionAnsweringModelOutput

import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
from tqdm import tqdm, tqdm_notebook

import time
import matplotlib.pyplot as plt

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# gpu 연산이 가능하면 'cuda:0', 아니면 'cpu' 출력
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device, torch.cuda.device_count()

(device(type='cuda', index=0), 1)

# Data Load

In [24]:
# https://huggingface.co/datasets/squad_kor_v1/blob/main/squad_kor_v1.py
# squad_kor_v2
from datasets import load_dataset
dataset = load_dataset('squad_kor_v1')
dataset, dataset['train'][0]

  0%|          | 0/2 [00:00<?, ?it/s]

(DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
}),
 {'answers': {'answer_start': [54], 'text': ['교향곡']},
  'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의

In [26]:
context_length, question_length, answer_location = [], [], []
for data in dataset['train'] :
  context_length.append(len(data['context']))
  question_length.append(len(data['question']))
  answer_location.append(context_length[-1] - data['answers']['answer_start'][0])    # answer_start가 1개라는 가정하에(잘못된 값이 없다는 가정) 문맥의 끝에서부터 위치 구하기
                                                                                    # (적게 차이나면 tokenizer에서 truncation때문에 잘릴 위험이 있다.) => truncation 제거!
print(f'max context_length : {max(context_length)}  min context_length : {min(context_length)}  mean context_length : {np.mean(np.array(context_length))}')
print(f'max question_length : {max(question_length)}  min question_length : {min(question_length)}  mean question_length : {np.mean(np.array(question_length))}')
print(f'max answer_location : {max(answer_location)}  min answer_location : {min(answer_location)}')

max context_length : 10012  min context_length : 348  mean context_length : 519.2681808399689
max question_length : 146  min question_length : 5  mean question_length : 33.79881470690483
max answer_location : 10005  min answer_location : 4


In [12]:
print(f"min answer_location : {np.argmin(answer_location)}")
print(f"min answer_idx's context : {dataset['train']['context'][np.argmin(np.array(answer_location))]}")
print(f"min answer_idx's context length : {len(dataset['train']['context'][np.argmin(np.array(answer_location))])}")

min answer_location : 1670
min answer_idx's context : 고조 사후, 혜제가 즉위하고 고황후가 실권을 잡아 나라를 다스렸다. 관영은 열후로서 고황후를 보좌했다. 태후가 죽자, 여록 등의 여씨 일족이 장안의 군사를 장악하고 있었고, 고조의 손자, 유비의 아들로 제나라의 왕을 지내는 유양은 불만을 품고 반란을 일으켜 여씨 세력을 쓸어버리고자 했다. 여씨 일족은 관영에게 군사를 맡겨 제나라의 반란을 진압하도록 했으나, 관영은 태위 주발, 승상 진평 등과 모의해 여씨를 치도록 작정했다. 그러기에 관영은 형양에 주둔하면서 소문을 퍼뜨려 제나라 군사가 진격을 멈추게 하고, 주발 등이 이 틈에 여씨들을 말갛게 쓸었다. 유양과 관영 모두 군사를 해산해 돌아갔고, 관영은 주발, 진평과 함께 고조의 아들 대왕을 황제로 세우니, 이가 곧 문제다.
min answer_idx's context length : 379


In [27]:
# 가장 긴 질문
dataset['train']['question'][np.argmax(np.array(question_length))]

'일본 메이지 대학교를 졸업하고 일본 문교사회국에서 근무한 경험이 있는 사람인 박철웅에게 설립동지회장직을 맡게하고, 대학설립을 준비하고자 도청이 소유한 차량을 사용하게 하고 도지사 명의로 사장과 군수에게 협조지시 공문을 보내는 등의 큰 역할을 한 사람은 누구인가?'

In [ ]:
text = tokenizer('하나의 교향곡을 쓰려는 뜻을 갖는다.', add_special_tokens=False).input_ids
answer = tokenizer('하나의 교향곡', add_special_tokens=False).input_ids
tokens = tokenizer(['1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다.', '이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다'], 
                   ['이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다', '이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다'], 
                   return_tensors='pt', truncation=True, padding=True, max_length=512)
text, answer, tokens

([3657, 2079, 19282, 2069, 1363, 2370, 2259, 936, 2069, 554, 2259, 2062, 18],
 [3657, 2079, 19282],
 {'input_ids': tensor([[    2, 13934,  2236,  2440, 27982,  2259, 21310,  2079, 11994,  3791,
           2069,  3790,  1508,  2088,   636,  3800,  2170,  3717,  2052,  9001,
           8345,  4642,  2200,  3689,  3657,  2079, 19282,  2069,  1363,  2370,
           2259,   936,  2069,   554,  2259,  2062,    18,     3,  8345,  4642,
           2200,  3689,  3657,  2079, 19282,  2069,  1363,  2370,  2259,   936,
           2069,   554,  2259,  2062,     3],
         [    2,  8345,  4642,  2200,  3689,  3657,  2079, 19282,  2069,  1363,
           2370,  2259,   936,  2069,   554,  2259,  2062,     3,  8345,  4642,
           2200,  3689,  3657,  2079, 19282,  2069,  1363,  2370,  2259,   936,
           2069,   554,  2259,  2062,     3,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,   

In [ ]:
tensorized_label = np.zeros(tokens.input_ids.shape)
print(answer)
for i in [0,1] :
  padding_start = (tokens['attention_mask'][i] == 1).nonzero()[-1].item()+1
  print(padding_start)
  tensorized_label[i, padding_start-len(text): padding_start-len(text)+len(answer)] = 1
  print(tensorized_label)
  print(tokens['input_ids'][i])
  print(tokens['input_ids'][i, padding_start-len(text): padding_start-len(text)+len(answer)])
  print('\n\n')

[3657, 2079, 19282]
55
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]]
tensor([    2, 13934,  2236,  2440, 27982,  2259, 21310,  2079, 11994,  3791,
         2069,  3790,  1508,  2088,   636,  3800,  2170,  3717,  2052,  9001,
         8345,  4642,  2200,  3689,  3657,  2079, 19282,  2069,  1363,  2370,
         2259,   936,  2069,   554,  2259,  2062,    18,     3,  8345,  4642,
         2200,  3689,  3657,  2079, 19282,  2069,  1363,  2370,  2259,   936,
         2069,   554,  2259,  2062,     3])
tensor([ 3657,  2079, 19282])



35
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.
  0. 0. 0. 0. 

In [ ]:
context = dataset['train']['context'][np.argmax(context_length)]
li = tokenizer(context).input_ids
# 정의되지않은 단어(한자 등)은 어떻게 할 것인가?//정답에 한문이 포함되어있기도 해서... 가지고 가야할 것 같다
tokenizer.decode([1]), li.count(1), dataset['train']['answers'][np.argmax(context_length)]

('[UNK]', 127, {'answer_start': [19], 'text': ['인길(仁吉)']})

In [5]:
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.question, self.context, self.answer_start, self.answer_text = self.make_dataset(dataset)

    def make_dataset(self, dataset):
        context, question, answer_start, answer_text = [], [], [], []
        for i, data in enumerate(dataset) :
          start = data['answers']['answer_start']
          if len(start) != 1 : # 답이 없을 때
            print(i, data)
            continue
          text, start = self.get_text(data['context'], start[0])
          answer_start.append(start)
          answer_text.append(data['answers']['text'])
          context.append(data['context'])
          question.append(data['question'])
        return question, context, answer_start, answer_text
        
    def __len__(self):
        return len(self.question)

    def __getitem__(self, idx):
        return self.question[idx], self.context[idx], self.answer_start[idx], self.answer_text[idx]

    def get_text(self, text, start_loc) :
        text_splited = text.split('. ')
        length_text_answer_idx = -1
        length_text = [0]

        for i, t in enumerate(text_splited) :
          length_text.append(length_text[-1]+len(t)+2)
          
          if length_text_answer_idx == -1 and start_loc < length_text[-1] :
            length_text_answer_idx = i-1
        length_text[-1] -= 2

        start, end = 0, len(length_text)-1
        # 이후에 question + context가 512 token 이하여야 한다.
        # context가 512일 때, Token indices sequence length is longer than the specified maximum sequence length for this model (946 > 512), 256일 때 (521 > 512), 
        # 정답이 context의 마지막에 위치할 수도 있기에(뒤에서 4번째에 위치하기도 함) context 길이 200으로 지정
        while length_text[end] - length_text[start] > 200 :  
            if start_loc - length_text[start] > length_text[end] - start_loc :
              start += 1
            else :
              end -= 1
        
        return text[length_text[start]:length_text[end]], start_loc - length_text[start]

In [28]:
def custom_collate_fn(batch):
    global tokenizer
    question_list, context_list, answer_start, answer_text, after_list = [], [], [], [], []

    for _question, _context, _start, _text in batch:
        question_list.append(_question)
        context_list.append(_context)
        after_list.append(_context[_start:])
        answer_start.append(_start)
        answer_text.append(_text)
    
    tensorized_input = tokenizer(    # 정답 잘림 방지를 위해 max_len과 truncation 제외?
        question_list, context_list,
        add_special_tokens=True,
        padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        max_length=512,
        truncation=True,
        return_tensors='pt'
    )
    # answer_start token의 위치를 찾기 위해 list로 반환
    after_text = tokenizer(
        after_list,
        add_special_tokens=False,
        return_tensors=None
    ).input_ids

    # answer text token만 변환
    answer_tokens = tokenizer(
        after_list,
        add_special_tokens=False,
        return_tensors=None
    ).input_ids

    tensorized_label = np.zeros(tensorized_input.input_ids.shape)    # input_ids만큼의 길이이고 0으로 된 array

    for i, zipped in enumerate(zip(after_text, answer_tokens)) :
        text, answer = zipped
        padding_start = (tensorized_input['attention_mask'][i] == 1).nonzero()[-1].item()+1
        tensorized_label[i, padding_start-len(text): padding_start-len(text)+len(answer)] = 1

    return tensorized_input, torch.from_numpy(tensorized_label)

In [7]:
def make_dataloader(dataset, tokenizer, batch_size, s='train') :
  dataloader = DataLoader(
      dataset,
      batch_size =batch_size,
      sampler = RandomSampler(dataset) if s == 'train' else SequentialSampler(dataset),
      collate_fn = custom_collate_fn
  )
  print(f'batch_size : {batch_size}')
  return dataloader

# 모델 설명


In [8]:
class CustomBertForQuestionAnswering(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.labels_type = [0, 1]   # 1은 정답 token의 위치(여러개 가능)
        self.num_labels = len(self.labels_type)
        self.bert = BertModel(config, add_pooling_layer=False)
        self.qa_output = nn.Linear(config.hidden_size, self.num_labels)
        # self.dropout = nn.Dropout(config.dripout_rate)

        self.post_init()
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        sequence_output = outputs[0]
        logits = self.qa_output(sequence_output)    # linear 통과해서 num_label로 분류
        return logits

# train

In [9]:
def initializer(train_dataloader, epochs=2, model_name='klue/bert-base', lr=4e-5, wd=4e-5):
    """
    모델, 옵티마이저, 스케쥴러 초기화
    """
    config = BertConfig.from_pretrained(model_name)
    config.max_length = 512
    model = CustomBertForQuestionAnswering(config)

    optimizer = AdamW(
        model.parameters(), # update 대상 파라미터를 입력
        lr=lr,    # 2e-5
        eps=1e-8,
        weight_decay=wd
    )
    
    total_steps = len(train_dataloader) * epochs
    print(f"Total train steps with {epochs} epochs: {total_steps}")

    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps = 0, # 여기서는 warmup을 사용하지 않는다.
        num_training_steps = total_steps
    )
    print(f'model_name : {model_name}, lr : {lr}, weight_decay : {wd}, epochs : {epochs}')
    return model, optimizer, scheduler

In [10]:
def save_checkpoint(path, model, optimizer, scheduler, epoch, loss, f1, model_name=''):
    file_name = f'{path}/epoch:{epoch}_loss:{loss:.4f}_f1:{f1:.4f}.ckpt'
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss' : loss,
            'f1' : f1
        }, 
        file_name
    )
    
    print(f"Saving epoch {epoch} checkpoint at {file_name}")

### train code

In [11]:
def train(model, optimizer, scheduler, train_dataloader, valid_dataloader=None, epochs=1):
  loss_fct = nn.MSELoss()
  em_fct = load_metric('exact_match')
  f1_fct = load_metric('f1')
  acc_fct = load_metric('accuracy')

  train_dict = {'loss' : [], 'f1' : []}
  valid_dict = {'loss' : [], 'f1' : [], 'em' : []}

  for epoch in range(epochs) :

    print(f"*****Epoch {epoch} Train Start*****")
    total_loss, total_acc, batch_acc, total_f1, batch_f1, batch_em, total_em, batch_loss, batch_count = 0,0,0,0,0,0,0,0,0
    
    model.train()
    model.to(device)
    
    for step, batch in enumerate(train_dataloader):
      batch_count+=1
      
      batch = tuple(item.to(device) for item in batch)
      batch_input, batch_label = batch
      
      model.zero_grad()
      
      outputs = model(**batch_input)  # forward
      outputs = torch.argmax(outputs, dim=2)
      loss = loss_fct(outputs, batch_label)
      loss.requires_grad_(True)

      batch_loss += loss.item()
      total_loss += loss.item()

      em = em_fct.compute(predictions=outputs.view([-1, 1]).squeeze(), references=batch_label.view([-1, 1]).squeeze())['exact_match']
      f1 = f1_fct.compute(predictions=outputs.view([-1, 1]).squeeze(), references=batch_label.view([-1, 1]).squeeze(), average='macro')['f1']
      accuracy = acc_fct.compute(predictions=outputs.view([-1, 1]).squeeze(), references=batch_label.view([-1, 1]).squeeze())['accuracy']

      batch_em += em
      total_em += em

      batch_f1 += f1
      total_f1 += f1
      
      batch_acc += accuracy
      total_acc += accuracy

      # backward -> 파라미터의 미분(gradient)를 자동으로 계산
      loss.backward()

      # gradient clipping 적용 
      clip_grad_norm_(model.parameters(), 1.0)
      
      # optimizer & scheduler 업데이트
      optimizer.step()
      scheduler.step()

      # 그래디언트 초기화
      model.zero_grad()

      if (step % 128 == 0 and step != 0):
          learning_rate = optimizer.param_groups[0]['lr']
          print(f"Epoch: {epoch}, Step : {step}, LR : {learning_rate:.10f}, Avg Loss : {batch_loss / batch_count:.4f}, f1 score : {batch_f1 / batch_count:.4f}")
          
          if (round(batch_f1 / batch_count, 5) == 0) and (round(learning_rate, 10) == 0) :
              print("Train Finished, learning_rate is 0 and train_f1 is 0")
              return train_dict, valid_dict

          batch_loss, batch_f1, batch_count = 0,0,0


    print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
    print(f"Epoch {epoch} Total Mean f1 : {total_f1/(step+1):.4f}")
    print(f"*****Epoch {epoch} Train Finish*****\n")

    train_dict['f1'].append(total_f1/(step+1))
    train_dict['loss'].append(total_loss/(step+1))
    # train_dict['em'].append(total_em/(step+1))
    
    if valid_dataloader is not None:
        print(f"*****Epoch {epoch} Valid Start*****")
        valid_loss, valid_em, valid_f1, valid_acc = validate(model, valid_dataloader, f1_fct, em_fct, acc_fct)
        print(f"Epoch {epoch} Valid Loss : {valid_loss:.4f} Valid f1 : {valid_f1:.4f} Valid em : {valid_em:.4f} Valid acc : {valid_acc:.4f}")
        print(f"*****Epoch {epoch} Valid Finish*****\n")

    valid_dict['f1'].append(valid_f1)
    valid_dict['loss'].append(valid_loss)
    valid_dict['em'].append(valid_em)
    if round(valid_f1, 4) == 0 :
        break
    # if before_loss > valid_loss :
    #     before_loss = valid_loss
    #     save_checkpoint("/content/drive/MyDrive/Colab Notebooks/nlp/qa", model, optimizer, scheduler, epoch, valid_loss, valid_f1, model_name)

    # elif before_f1 < valid_f1  :
    #     before_f1 = valid_f1
    #     save_checkpoint("/content/drive/MyDrive/Colab Notebooks/nlp/qa", model, optimizer, scheduler, epoch, valid_loss, valid_f1, model_name)

  print("Train Finished")
  return train_dict, valid_dict

### validation code

In [12]:
def validate(model, valid_dataloader, f1_fct, em_fct, acc_fct):
    loss_fct = nn.MSELoss()
    model.eval()
    model.to(device)
    
    total_loss, total_em, total_f1, total_acc= 0,0, 0, 0
        
    for step, batch in enumerate(valid_dataloader):
        
        batch = tuple(item.to(device) for item in batch)
            
        batch_input, batch_label = batch
            
        # gradient 계산하지 않음
        with torch.no_grad():
            outputs = model(**batch_input)
            outputs = torch.argmax(outputs, dim=2)

        loss = loss_fct(outputs, batch_label)
        
        em = em_fct.compute(predictions=outputs.view([-1, 1]).squeeze(), references=batch_label.view([-1, 1]).squeeze())['exact_match']
        f1 = f1_fct.compute(predictions=outputs.view([-1, 1]).squeeze(), references=batch_label.view([-1, 1]).squeeze())['f1']
        acc = acc_fct.compute(predictions=outputs.view([-1, 1]).squeeze(), references=batch_label.view([-1, 1]).squeeze())['accuracy']
        
        total_loss += loss.item()
        total_f1 += f1
        total_em += em
        total_acc += acc

    total_loss = total_loss/(step+1)
    total_em = total_em/(step+1)
    total_f1 = total_f1/(step+1)
    total_acc = total_acc/(step+1)
    return total_loss, total_em, total_f1, total_acc

### draw_plot

In [13]:
# loss와 f1-score의 변화를 epoch마다 보기 위한 plot
def draw_plot(train_dict, valid_dict, i) :
  print('green is loss, gray is f1')
  plt.subplot(1, 2, 1)
  plt.xlabel('Epochs')
  plt.title('Loss and F1 of Train data')
  x_values= [n for n in range(len(train_dict['loss']))]
  plt.plot(x_values, train_dict['loss'], color='green', marker='o')  # loss
  plt.plot(x_values, train_dict['f1'], color='#AAAAAA', marker='*')  # f1

  plt.subplot(1, 2, 2)
  plt.xlabel('Epochs')
  plt.title('Loss and F1 of Validation data')
  x_values= [n for n in range(len(valid_dict['loss']))]
  plt.plot(x_values, valid_dict['loss'], color='green', marker='o')  # loss
  plt.plot(x_values, valid_dict['f1'], color='#AAAAAA', marker='*')  # f1

  plt.show()
  plt.savefig(f'figure_{i}.png')

In [14]:
model_name = 'klue/bert-base'   # 다시 설정 필요
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [29]:
train_dataset = CustomDataset(dataset['train'])
valid_dataset = CustomDataset(dataset['validation'])

del dataset

In [30]:
import gc
gc.collect()

397

In [31]:
train_dataloader = make_dataloader(train_dataset, model_name, 16, 'train')
valid_dataloader = make_dataloader(valid_dataset, model_name, 8, 'valid')



batch_size : 16
batch_size : 8


In [32]:

learning_rate = 5e-5
weight_decay = 4e-5
model, optimizer, scheduler = initializer(train_dataloader, 4, model_name, learning_rate, weight_decay)
start = time.time()

Total train steps with 4 epochs: 15104
model_name : klue/bert-base, lr : 5e-05, weight_decay : 4e-05, epochs : 4


In [33]:
# del train_dataset
# del valid_dataset

In [34]:
import gc
gc.collect()

109

In [35]:
import warnings
warnings.filterwarnings(action='ignore')

In [36]:
train_dict, valid_dict = train(model, optimizer, scheduler, train_dataloader, valid_dataloader, 4)
end = time.time()
print(f"time : {(end - start)//60}분 {(end - start)%60}초")

# draw_plot(train_dict, valid_dict, 0)

*****Epoch 0 Train Start*****
Epoch: 0, Step : 128, LR : 0.0000495730, Avg Loss : 0.4334, f1 score : 0.5426
Epoch: 0, Step : 256, LR : 0.0000491492, Avg Loss : 0.4334, f1 score : 0.5418
Epoch: 0, Step : 384, LR : 0.0000487255, Avg Loss : 0.4313, f1 score : 0.5425
Epoch: 0, Step : 512, LR : 0.0000483018, Avg Loss : 0.4328, f1 score : 0.5416
Epoch: 0, Step : 640, LR : 0.0000478780, Avg Loss : 0.4313, f1 score : 0.5429
Epoch: 0, Step : 768, LR : 0.0000474543, Avg Loss : 0.4317, f1 score : 0.5432
Epoch: 0, Step : 896, LR : 0.0000470306, Avg Loss : 0.4294, f1 score : 0.5435
Epoch: 0, Step : 1024, LR : 0.0000466069, Avg Loss : 0.4310, f1 score : 0.5436
Epoch: 0, Step : 1152, LR : 0.0000461831, Avg Loss : 0.4325, f1 score : 0.5432
Epoch: 0, Step : 1280, LR : 0.0000457594, Avg Loss : 0.4311, f1 score : 0.5429
Epoch: 0, Step : 1408, LR : 0.0000453357, Avg Loss : 0.4289, f1 score : 0.5443
Epoch: 0, Step : 1536, LR : 0.0000449119, Avg Loss : 0.4324, f1 score : 0.5424
Epoch: 0, Step : 1664, LR : 0

KeyboardInterrupt: ignored

[Pytorch Model 일부 Layer만 Freeze 하기](https://soyoung97.github.io/pytorch-model-%EC%9D%BC%EB%B6%80-layer%EB%A7%8C-freeze-%ED%95%98%EA%B8%B0/)